In [160]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

#import dataset from weather_data.csv file
data = pd.read_csv("D:\weather_data.csv", sep = ";")
data[:]


,time,month,temperature,feelslike,wind,direction,gust,cloud,humidity,precipitation,pressure,weather
0,0:00,1,24.0,28.0,8.0,ENE,12.0,4.0,86.0,0.0,1012.0,Clear
1,3:00,1,23.0,27.0,8.0,NE,10.0,4.0,88.0,0.0,1011.0,Clear
2,6:00,1,23.0,26.0,8.0,NNE,11.0,7.0,85.0,0.0,1012.0,Sunny
3,9:00,1,28.0,33.0,11.0,NNE,13.0,6.0,64.0,0.0,1012.0,Sunny
4,12:00,1,31.0,35.0,10.0,ENE,12.0,62.0,53.0,0.0,1010.0,Partly cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...
8507,9:00,11,29.0,32.0,4.0,ENE,5.0,17.0,68.0,0.0,1012.0,Partly cloudy
8508,12:00,11,33.0,37.0,5.0,ENE,5.0,15.0,52.0,0.0,1011.0,Partly cloudy
8509,15:00,11,34.0,38.0,3.0,ENE,4.0,19.0,50.0,0.0,1009.0,Partly cloudy
8510,18:00,11,31.0,34.0,6.0,SE,8.0,19.0,65.0,0.0,1009.0,Partly cloudy


In [161]:
num_data = data[['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'humidity', 'pressure']]
cat_data = data[['time', 'month', 'direction', 'weather']]

In [162]:
sc = MinMaxScaler()
ohe = OneHotEncoder()
num_data = sc.fit_transform(num_data)
cat_data = ohe.fit_transform(cat_data).toarray()
data = np.concatenate((cat_data, num_data), axis = 1)

n_future = 8
n_past = 56

x_train, y_train = [], []
for i in range(n_past, round(len(data)*0.7)):
    x_train.append(data[i - n_past : i, :])
    y_train.append(data[i : i + n_future , 0 ])

x_train, y_train = np.array(x_train), np.array(y_train)


In [163]:
from keras.models import Sequential
from keras.layers import LSTM, Dense ,Dropout
from keras.layers import Bidirectional
# Fitting RNN to training set using Keras Callbacks.
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2]))))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 50 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 50 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 8,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])
regressor.fit(x_train, y_train, epochs=10,batch_size=32)

Epoch 1/10
185/185 [==============================] - 26s 60ms/step - loss: 0.1005 - accuracy: 0.2767
Epoch 2/10
185/185 [==============================] - 11s 61ms/step - loss: 0.0128 - accuracy: 0.9959
Epoch 3/10
185/185 [==============================] - 11s 59ms/step - loss: 0.0069 - accuracy: 0.9998
Epoch 4/10
185/185 [==============================] - 11s 60ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 5/10
185/185 [==============================] - 10s 55ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 6/10
185/185 [==============================] - 10s 55ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 7/10
185/185 [==============================] - 10s 57ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 8/10
185/185 [==============================] - 11s 61ms/step - loss: 0.0121 - accuracy: 0.9671
Epoch 9/10
185/185 [==============================] - 11s 60ms/step - loss: 0.0060 - accuracy: 0.9983
Epoch 10/10
185/185 [==============================] - 10s 55ms/step - loss: 0.004

In [172]:
x_test = data[len(data)-n_past-n_future:len(data)-n_future, :]
x_test = np.array(x_test)
x_test = np.reshape(x_test, (1, x_test.shape[0], x_test.shape[1]))
y_pred = regressor.predict(x_test)


1/1 [==============================] - 0s 25ms/step
